In [5]:
import pandas as pd
import numpy
from pandas import DataFrame as df

metadata=pd.read_csv("/content/drive/My Drive/IMDB-Movie-Data.csv")
metadata



,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0
5,6,The Great Wall,"Action,Adventure,Fantasy",European mercenaries searching for black powde...,Yimou Zhang,"Matt Damon, Tian Jing, Willem Dafoe, Andy Lau",2016,103,6.1,56036,45.13,42.0
6,7,La La Land,"Comedy,Drama,Music",A jazz pianist falls for an aspiring actress i...,Damien Chazelle,"Ryan Gosling, Emma Stone, Rosemarie DeWitt, J....",2016,128,8.3,258682,151.06,93.0
7,8,Mindhorn,Comedy,A has-been actor best known for playing the ti...,Sean Foley,"Essie Davis, Andrea Riseborough, Julian Barrat...",2016,89,6.4,2490,NaN,71.0
8,9,The Lost City of Z,"Action,Adventure,Biography","A true-life drama, centering on British explor...",James Gray,"Charlie Hunnam, Robert Pattinson, Sienna Mille...",2016,141,7.1,7188,8.01,78.0
9,10,Passengers,"Adventure,Drama,Romance",A spacecraft traveling to a distant colony pla...,Morten Tyldum,"Jennifer Lawrence, Chris Pratt, Michael Sheen,...",2016,116,7.0,192177,100.01,41.0



**1. 인기도가 반영되어 있지 않아서 Weighted Rating을 새로 생성**


*   10명이 평가한 9점과 10000명이 평가한 8.9점
적은 평가자가 있는 경우 한쪽으로 평점이 쏠릴 가능성이 높지만,
평가자가 많아짐에 따라 정규화가 이루어지면서 평점이 실제 영화 품질을 반영
m 결정을 위해 90% percentile 사용












In [7]:
C=metadata['Rating'].mean()
print(C)

#90퍼센트 값
m=metadata['Votes'].quantile(0.9)
print(m)

q_movies=metadata.copy().loc[metadata['Votes']>=m]
print(q_movies.shape)

#영화 평점 점수 기준 수정

def WR(x,m=m,C=C):
    v=x['Votes']
    R=x['Rating']
    return (v/(v+m)*R) + (m/(m+v)*C)

6.723200000000003
406251.10000000003
(100, 12)


In [0]:
metadata['score']=metadata.apply(WR,axis=1)

print(metadata['Votes'])

0      757074
1      485820
2      157606
3       60545
4      393727
5       56036
6      258682
7        2490
8        7188
9      192177
10     232072
11      93103
12     323118
13     118151
14       8612
15     120259
16     211760
17     150823
18     102061
19     340798
20      19053
21     134213
22       1115
23      38552
24     127553
25        222
26      76193
27        523
28      66540
29     112813
        ...  
970     37060
971    193491
972     64513
973    108544
974      6753
975     20966
976     31634
977      2229
978     10658
979    108772
980     12048
981     27312
982     95172
983    112729
984     21405
985     87904
986     88000
987     33688
988     63785
989     67637
990    129708
991    102697
992     45419
993    140900
994    164088
995     27585
996     73152
997     70699
998      4881
999     12435
Name: Votes, Length: 1000, dtype: int64


# 정렬 및 결과 확인 

In [0]:
col=['Title','Rating','Votes','score']
rs_original=metadata.sort_values("Rating",ascending=False)[col]
rs_new=metadata.sort_values('score',ascending=False)[col]

In [0]:
rs_original.head()

,Title,Rating,Votes,score
54,The Dark Knight,9.0,1791916,8.579216
80,Inception,8.8,1583625,8.376003
117,Dangal,8.8,48969,6.946606
36,Interstellar,8.6,1047747,8.075617
96,Kimi no na wa,8.6,34110,6.868575


In [0]:
rs_new.head()

,Title,Rating,Votes,score
54,The Dark Knight,9.0,1791916,8.579216
80,Inception,8.8,1583625,8.376003
36,Interstellar,8.6,1047747,8.075617
124,The Dark Knight Rises,8.5,1222645,8.056861
99,The Departed,8.5,937414,7.962793


# 영와 description 유사도 계산 

In [0]:
metadata['Description'].head()


0    A group of intergalactic criminals are forced ...
1    Following clues to the origin of mankind, a te...
2    Three girls are kidnapped by a man with a diag...
3    In a city of humanoid animals, a hustling thea...
4    A secret government agency recruits some of th...
Name: Description, dtype: object

# Descriptin 의 TF-IDF 계산  : null값이 존재함에 따라 fillna 적용

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer   #단어를 갯수 그대로 카운트하지 않고 모든 문서에 
                                                            #공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법.

tfidf=TfidfVectorizer(stop_words='english')                      #무시할 단어

metadata.Descroption=metadata.Description.fillna('')

tfidf_matrix=tfidf.fit_transform(metadata.Description)
tfidf_matrix.shape

(1000, 5667)

In [0]:
#TF-IDF 유사도 계산

from  sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
cosine_sim.shape

(1000, 1000)

In [0]:
#추천 리스트 생성
#영화 타이틀을 인덱스화
indices=pd.Series(metadata.index, index=metadata['Title']).drop_duplicates()

In [0]:
indices

Title
Guardians of the Galaxy                      0
Prometheus                                   1
Split                                        2
Sing                                         3
Suicide Squad                                4
The Great Wall                               5
La La Land                                   6
Mindhorn                                     7
The Lost City of Z                           8
Passengers                                   9
Fantastic Beasts and Where to Find Them     10
Hidden Figures                              11
Rogue One                                   12
Moana                                       13
Colossal                                    14
The Secret Life of Pets                     15
Hacksaw Ridge                               16
Jason Bourne                                17
Lion                                        18
Arrival                                     19
Gold                                        20
Manches

In [0]:
def rs(Title, n=10, sim=cosine_sim):
    idx=indices[Title]
    sim_score= list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score,key=lambda x:x[1], reverse=True)
    sim_score = sim_score[1:(n+1)]
    
    movie_index=[i[0] for i in sim_score]
    
    return metadata.Title.iloc[movie_index]

In [0]:
rs('The Dark Knight Rises')

54              The Dark Knight
144            Django Unchained
522                      Viking
471            Marie Antoinette
254               The Conjuring
920                   Centurion
253    The Amazing Spider-Man 2
604       The Rise of the Krays
615                Bastille Day
915                     Goksung
Name: Title, dtype: object